In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [ ]:
!pip install flask-cloudflared
!pip install xformers

In [ ]:
!apt-get update -y && apt-get upgrade -y
!apt-get install -y git ffmpeg libgl1 libglib2.0-0
!pip install comfy-cli
!yes | comfy install


In [ ]:
!cd /root/comfy/ComfyUI/custom_nodes && git clone https://github.com/rgthree/rgthree-comfy.git

In [ ]:
# FLUX TEXT TO IMAGE
# Diffusion model
!mkdir -p /root/comfy/ComfyUI/models/diffusion_models
!cd /root/comfy/ComfyUI/models/diffusion_models && wget -O CyberRealistic_Flux_V1.5.safetensors "https://huggingface.co/cyberdelia/CyberRealisticFlux/resolve/main/CyberRealistic_Flux_V1.5.safetensors?download=true"
# Text encoders
!mkdir -p /root/comfy/ComfyUI/models/text_encoders
!cd /root/comfy/ComfyUI/models/text_encoders && wget -O clip_l.safetensors https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors
!cd /root/comfy/ComfyUI/models/text_encoders && wget -O t5xxl_fp16.safetensors https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors

# VAE
!mkdir -p /root/comfy/ComfyUI/models/vae
!cd /root/comfy/ComfyUI/models/vae && wget -O ae.safetensors https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors

# LoRAs
#!mkdir -p /root/comfy/ComfyUI/models/loras
# !cd /root/comfy/ComfyUI/models/loras && wget --content-disposition "# Link here"


In [ ]:
import subprocess
import threading
import time

# ==============================================================================
# Step 1: Download and prepare the cloudflared executable
# ==============================================================================
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

# ==============================================================================
# Step 2: Define and run your ComfyUI app on port 8188
# ==============================================================================
PORT = 8188

def run_comfy():
  # This function runs the ComfyUI python script using subprocess
  subprocess.run(
      ["python", "main.py", "--listen", "0.0.0.0", "--port", str(PORT)],
      cwd="/root/comfy/ComfyUI"  # Specifies the directory to run the command in
  )

# Start the ComfyUI server in a background thread
# This allows our script to continue to the next step
threading.Thread(target=run_comfy, daemon=True).start()

# Wait a few seconds to allow the ComfyUI server to start up
print("⏳ Waiting for ComfyUI to start...")
time.sleep(10)
print("✅ ComfyUI should be running now.")

# ==============================================================================
# Step 3: Create the Cloudflare Tunnel to forward port 8188
# ==============================================================================
# This command creates a public URL and forwards all traffic to your
# ComfyUI instance running on the specified local port.
# This command will keep running, holding the Colab cell active.
!./cloudflared tunnel --url http://localhost:{PORT}